In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder


In [2]:
movies = pd.read_csv("dataset/small_dataset/movies_full_2.csv")
ratings = pd.read_csv("dataset/small_dataset/ratings.csv")
tags = pd.read_csv("dataset/small_dataset/tags.csv")
tags.drop(columns=['userId', 'timestamp'], inplace=True)

In [3]:
def create_weighted_rating_tags_df(movies_df, ratings_df, tags_df):
    movies_rating_user_df = pd.merge(movies_df, ratings_df, on="movieId", how="inner")

    movies_rating_df = movies_rating_user_df[['movieId', 'title', 'rating', 'genres', 'year', 'url']].groupby(['movieId', 'title', 'genres', 'year', 'url'])['rating'].agg(['count', 'mean']).round(1)
    movies_rating_df.sort_values('count', ascending=False, inplace=True)
    movies_rating_df.rename(columns={'count' : 'Num_ratings', 'mean': 'Average_rating'}, inplace=True)

    C = round(ratings_df['rating'].mean(), 2)
    m = 500
    movies_rating_df['Bayesian_rating'] = (movies_rating_df['Num_ratings'] / (movies_rating_df['Num_ratings'] + m)) * movies_rating_df['Average_rating'] + (m / (movies_rating_df['Num_ratings'] + m)) * C
    movies_rating_df.drop(columns='Average_rating', inplace=True)
    movies_rating_df.rename(columns={'Num_ratings' : 'count', 'Bayesian_rating' : 'weighted_rating'}, inplace=True)
    movies_rating_df.reset_index(inplace=True)
    

    movies_rating_tags_df = pd.merge(movies_rating_df, tags_df, how='left', on='movieId')
    movies_rating_tags_df['tag'] = movies_rating_tags_df['tag'].fillna(value='')
    movies_rating_tags_df = movies_rating_tags_df.groupby(['movieId', 'title', 'genres', 'year', 'url', 'count', 'weighted_rating'])['tag'].apply(list).reset_index()
    movies_rating_tags_df['genres'] = movies_rating_tags_df['genres'].str.split('|')
    movies_rating_tags_df['tag'] = movies_rating_tags_df['tag'].apply(lambda x: [] if x == [float('nan')] else x)
    movies_rating_tags_df.sort_values(by='weighted_rating', ascending=False, inplace=True)
    return movies_rating_tags_df


movies_rating_tags_df = create_weighted_rating_tags_df(movies, ratings, tags)
movies_rating_tags_df.head(20)

,movieId,title,genres,year,url,count,weighted_rating,tag
276,318,"Shawshank Redemption, The","[Crime, Drama]",1994.0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,317,3.849204,"[prison, Stephen King, wrongful imprisonment, ..."
313,356,Forrest Gump,"[Comedy, Drama, Romance, War]",1994.0,https://m.media-amazon.com/images/M/MV5BNWIwOD...,329,3.777805,"[shrimp, Vietnam, bubba gump shrimp, lieutenan..."
256,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994.0,https://m.media-amazon.com/images/M/MV5BNGNhMD...,307,3.766295,"[good dialogue, great soundtrack, non-linear, ..."
509,593,"Silence of the Lambs, The","[Crime, Horror, Thriller]",1991.0,https://m.media-amazon.com/images/M/MV5BNjNhZT...,279,3.750706,"[Hannibal Lector, disturbing, drama, gothic, p..."
1932,2571,"Matrix, The","[Action, Sci-Fi, Thriller]",1999.0,https://m.media-amazon.com/images/M/MV5BNzQzOT...,278,3.750129,"[martial arts, sci-fi, alternate universe, phi..."
2217,2959,Fight Club,"[Action, Crime, Drama, Thriller]",1999.0,https://m.media-amazon.com/images/M/MV5BMmEzNT...,218,3.742897,"[dark comedy, psychology, thought-provoking, t..."
224,260,Star Wars: Episode IV - A New Hope,"[Action, Adventure, Sci-Fi]",1977.0,https://m.media-amazon.com/images/M/MV5BNzVlY2...,251,3.733955,"[classic, space action, action, sci-fi, EPIC, ..."
657,858,"Godfather, The","[Crime, Drama]",1972.0,https://m.media-amazon.com/images/M/MV5BM2MyNj...,192,3.721965,[Mafia]
460,527,Schindler's List,"[Drama, War]",1993.0,https://m.media-amazon.com/images/M/MV5BNDE4OT...,220,3.713889,"[moving, thought-provoking, Holocaust, based o..."
892,1196,Star Wars: Episode V - The Empire Strikes Back,"[Action, Adventure, Sci-Fi]",1980.0,https://m.media-amazon.com/images/M/MV5BYmU1ND...,211,3.707736,"[I am your father, space, space opera, classic..."


In [4]:
print(movies_rating_tags_df.iloc[2137])

movieId                                                       157110
title                           00 Schneider - Jagd auf Nihil Baxter
genres                                               [Comedy, Crime]
year                                                          1994.0
url                https://m.media-amazon.com/images/M/MV5BOTQzMz...
count                                                              1
weighted_rating                                             3.501996
tag                                                               []
Name: 8954, dtype: object


# Knowledge-based

In [5]:
def knowledge_based_recommendation(df, user_preferences):
    # Filter movies based on user's preferred genres
    recommended_movies = df[df['genres'].apply(lambda x: any(genre in user_preferences['preferred_genres'] for genre in x))]
    
    # Remove movies with disliked genres
    if user_preferences['disliked_genres']:
        recommended_movies = recommended_movies[~recommended_movies['genres'].apply(lambda x: any(genre in user_preferences['disliked_genres'] for genre in x))]
    
    # Return only the movie IDs of the top 3 recommendations
    return recommended_movies.head(5)['movieId'].tolist()

user_preferences = {'preferred_genres': ['Action', 'Adventure'], 'disliked_genres': ['Crime']}
k_recommendations_ids = knowledge_based_recommendation(movies_rating_tags_df, user_preferences)
print(k_recommendations_ids)



[2571, 260, 1196, 1198, 4993]


In [6]:
print(movies_rating_tags_df.loc[k_recommendations_ids])

      movieId                       title             genres    year  \
2571     3450              Grumpy Old Men           [Comedy]  1993.0   
260       301  Picture Bride (Bijo photo)   [Drama, Romance]  1994.0   
1196     1600             She's So Lovely   [Drama, Romance]  1997.0   
1198     1602          Leave It to Beaver           [Comedy]  1997.0   
4993     7815                True Stories  [Comedy, Musical]  1986.0   

                                                    url  count  \
2571  https://m.media-amazon.com/images/M/MV5BMzNiYz...     29   
260   https://m.media-amazon.com/images/M/MV5BY2M2NG...      1   
1196  https://m.media-amazon.com/images/M/MV5BZmU1ZT...      2   
1198  https://m.media-amazon.com/images/M/MV5BYTNkMW...      4   
4993  https://m.media-amazon.com/images/M/MV5BYWU0Nj...      1   

      weighted_rating tag  
2571         3.489036  []  
260          3.500998  []  
1196         3.500000  []  
1198         3.486508  []  
4993         3.502994  []  


# Content-based

In [7]:
def content_based_recommendation(df, user_preferences):
    # Combine genres and tags to create a single text representation for each movie
    df['features'] = df['genres'] + df['tag']
    df['features'] = df['features'].apply(lambda x: ' '.join(x))
    
    # Create TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    
    # Fit and transform the TF-IDF vectorizer
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['features'])
    
    # Calculate cosine similarity between movies
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    # Get indices of movies similar to the user's liked movies
    similar_movies_indices = []
    for movie_title in user_preferences['liked_movies']:
        idx = df.index[df['title'] == movie_title][0]
        similar_movies_indices.extend(cosine_sim[idx].argsort()[-2:-11:-1])  # Get top 10 similar movies
    
    # Remove duplicates and movies already liked by the user
    similar_movies_indices = list(set(similar_movies_indices) - set(df.index[df['title'].isin(user_preferences['liked_movies'])]))
    
    # # Return top 3 recommended movies
    # recommended_movies = df.loc[similar_movies_indices[:3]]
    # return recommended_movies
    return similar_movies_indices[:5]

# User Input (User's liked movies)
user_preferences = {'liked_movies': ['Shawshank Redemption, The', 'Toy Story']}

# Get content-based recommendations with genres and tags
content_based_recommendations_ids = content_based_recommendation(movies_rating_tags_df, user_preferences)
print(content_based_recommendations_ids)

print(movies_rating_tags_df.loc[content_based_recommendations_ids])

# # Display recommended movies
# print("Content-Based Recommendations with Genres and Tags:")
# print(content_based_recommendations[['title', 'genres', 'tag', 'year']])


[1089, 841, 393, 3593, 204]
      movieId                                        title  \
1089     1423                             Hearts and Minds   
841      1116             Single Girl, A (Fille seule, La)   
393       453                            For Love or Money   
3593     4957                                Sudden Impact   
204       238  Far From Home: The Adventures of Yellow Dog   

                     genres    year  \
1089                [Drama]  1996.0   
841                 [Drama]  1995.0   
393       [Comedy, Romance]  1993.0   
3593      [Crime, Thriller]  1983.0   
204   [Adventure, Children]  1995.0   

                                                    url  count  \
1089  https://m.media-amazon.com/images/M/MV5BYzMzMG...      1   
841   https://m.media-amazon.com/images/M/MV5BYWI4OW...      1   
393   https://m.media-amazon.com/images/M/MV5BYjg1NG...      5   
3593  https://m.media-amazon.com/images/M/MV5BYTA2Nz...      2   
204   https://m.media-amazon.com/i

# collaborative-filtering

In [8]:
# Function to create the utility matrix
def create_utility_matrix(df):
    utility_matrix = df.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    return utility_matrix

# Function for collaborative filtering recommendation using kNN
def collaborative_filtering_recommendation(df, user_id, k, num_recommendations):
    # Create the utility matrix
    utility_matrix = create_utility_matrix(df)
    
    # Map user IDs to indices
    user_indices = {user_id: idx for idx, user_id in enumerate(utility_matrix.index)}
    
    # Check if the given user ID exists
    if user_id not in user_indices:
        print("User ID does not exist.")
        return []
    
    # Calculate cosine similarity between users
    user_similarity = cosine_similarity(utility_matrix)
    
    # Find k nearest neighbors for the target user
    knn = NearestNeighbors(n_neighbors=k, metric='cosine')
    knn.fit(user_similarity)
    _, indices = knn.kneighbors([user_similarity[user_indices[user_id]]])
    
    # Get the ratings of the nearest neighbors
    neighbor_ratings = utility_matrix.iloc[indices[0]]
    
    # Calculate the average rating for each item
    item_ratings = neighbor_ratings.mean(axis=0)
    
    # Filter out items already rated by the target user
    user_ratings = utility_matrix.loc[user_id]
    recommended_items = item_ratings[user_ratings == 0].sort_values(ascending=False)

    return recommended_items.index.tolist()[:num_recommendations]  

# User Input (User ID and number of nearest neighbors)
user_id = 200010
k_neighbors = 3
num_recommendations = 5

# Get collaborative filtering recommendations using kNN
collaborative_filtering_recommendations_ids = collaborative_filtering_recommendation(ratings, user_id, k_neighbors, num_recommendations)

# Display recommended movie IDs
print("Collaborative Filtering Recommendations:", collaborative_filtering_recommendations_ids)

# Filter the movies DataFrame based on recommended movie IDs
collaborative_filtering_recommendations_df = movies_rating_tags_df[movies_rating_tags_df['movieId'].isin(collaborative_filtering_recommendations_ids)]

# Display recommended movies
print("Collaborative Filtering Recommendations:")
print(collaborative_filtering_recommendations_df)

Collaborative Filtering Recommendations: [594, 7025, 3996, 750, 2174]
Collaborative Filtering Recommendations:
      movieId                                              title  \
600       750  Dr. Strangelove or: How I Learned to Stop Worr...   
2970     3996   Crouching Tiger, Hidden Dragon (Wo hu cang long)   
510       594                    Snow White and the Seven Dwarfs   
4682     7025                                  Midnight Clear, A   
1621     2174                                        Beetlejuice   

                                              genres    year  \
600                                    [Comedy, War]  1964.0   
2970                        [Action, Drama, Romance]  2000.0   
510   [Animation, Children, Drama, Fantasy, Musical]  1937.0   
4682                                    [Drama, War]  1992.0   
1621                               [Comedy, Fantasy]  1988.0   

                                                    url  count  \
600   https://m.media-amazon.

# Hybrid-filtering

In [9]:
def hybrid_based_recommendation(knowledge_recommendations_ids, content_recommendations_ids, collaborative_recommendations_ids):

    knowledge_weight = 1
    content_weight = 1
    collaborative_weight = 1

     # Calculate the number of movies to be recommended from each technique
    num_movies_per_technique = 5
    
    # Apply weights to each technique's recommendations
    weighted_knowledge_recommendations = knowledge_recommendations_ids[:num_movies_per_technique] * knowledge_weight
    weighted_content_recommendations = content_recommendations_ids[:num_movies_per_technique] * content_weight
    weighted_collaborative_recommendations = collaborative_recommendations_ids[:num_movies_per_technique] * collaborative_weight
    
    # Combine recommendations from all techniques
    combined_recommendations = weighted_knowledge_recommendations + weighted_content_recommendations + weighted_collaborative_recommendations

    combined_recommendations = list(set(combined_recommendations))

    
    # Ensure the total number of recommendations is not more than 10
    combined_recommendations = combined_recommendations[:10]
    
    return combined_recommendations
    
recommendations_hybrid = hybrid_based_recommendation(k_recommendations_ids,content_based_recommendations_ids, collaborative_filtering_recommendations_ids)
print(recommendations_hybrid)


[4993, 1089, 260, 841, 393, 2571, 1196, 3593, 1198, 204]


In [10]:
print(movies_rating_tags_df.loc[recommendations_hybrid])

      movieId                                        title  \
4993     7815                                 True Stories   
1089     1423                             Hearts and Minds   
260       301                   Picture Bride (Bijo photo)   
841      1116             Single Girl, A (Fille seule, La)   
393       453                            For Love or Money   
2571     3450                               Grumpy Old Men   
1196     1600                              She's So Lovely   
3593     4957                                Sudden Impact   
1198     1602                           Leave It to Beaver   
204       238  Far From Home: The Adventures of Yellow Dog   

                     genres    year  \
4993      [Comedy, Musical]  1986.0   
1089                [Drama]  1996.0   
260        [Drama, Romance]  1994.0   
841                 [Drama]  1995.0   
393       [Comedy, Romance]  1993.0   
2571               [Comedy]  1993.0   
1196       [Drama, Romance]  1997.0   
3593 

# Deep learning

In [11]:
#
movies_rating_tags_user = pd.merge(movies_rating_tags_df, ratings, on='movieId', how='left')
movies_rating_tags_user.drop(columns=['timestamp', 'url', 'genres', 'tag'], inplace=True)
movies_rating_tags_user.head(10)
num_rows = len(movies_rating_tags_user)
print("Number of rows:", num_rows)

Number of rows: 100181


In [12]:
# Encode categorical variables
label_encoders = {}
for column in ['title', 'features']:
    label_encoders[column] = LabelEncoder()
    movies_rating_tags_user[column] = label_encoders[column].fit_transform(movies_rating_tags_user[column])

# Split data into train and test sets
train_df, test_df = train_test_split(movies_rating_tags_user, test_size=0.2, random_state=42)

# Define the number of unique users and movies
num_users = movies_rating_tags_user['userId'].nunique()
num_movies = movies_rating_tags_user['movieId'].nunique()

# Define the input layers
user_input = Input(shape=(1,), name='user_input')
movie_input = Input(shape=(1,), name='movie_input')
year_input = Input(shape=(1,), name='year_input')
count_input = Input(shape=(1,), name='count_input')
weighted_rating_input = Input(shape=(1,), name='weighted_rating_input')
features_input = Input(shape=(1,), name='features_input')

# Embedding layers
user_emb = Embedding(input_dim=num_users, output_dim=50, input_length=1)(user_input)
movie_emb = Embedding(input_dim=num_movies, output_dim=50, input_length=1)(movie_input)

# Flatten the embeddings
user_flatten = Flatten()(user_emb)
movie_flatten = Flatten()(movie_emb)

# Concatenate all input features
concat = Concatenate()([user_flatten, movie_flatten, year_input, count_input, weighted_rating_input, features_input])

# Dense layers
dense1 = Dense(100, activation='relu')(concat)
dense2 = Dense(50, activation='relu')(dense1)
output = Dense(1)(dense2)

# Define the model
model = Model(inputs=[user_input, movie_input, year_input, count_input, weighted_rating_input, features_input], outputs=output)

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

# Train the model
model.fit([train_df['userId'], train_df['movieId'], train_df['year'], train_df['count'], train_df['weighted_rating'], train_df['features']],
          train_df['rating'], batch_size=64, epochs=5, validation_split=0.2)

# Evaluate the model
mse = model.evaluate([test_df['userId'], test_df['movieId'], test_df['year'], test_df['count'], test_df['weighted_rating'], test_df['features']], test_df['rating'])
print("Test MSE:", mse)

# Make recommendations (predict ratings for new user-movie pairs)
# For example, model.predict([new_user_id, new_movie_id, new_year, new_count, new_weighted_rating, new_features])

Epoch 1/5



InvalidArgumentError: Graph execution error:

Detected at node model/embedding_1/embedding_lookup defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\João\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\João\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\João\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\João\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\João\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\João\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\

# Debugging


In [ ]:
user_id = 5

# Filter ratings for the specified user
ratings_user = ratings[ratings['userId'] == user_id]

# Get the IDs of the last three movies seen by the user
titles_ids = ratings_user['movieId'].tail(3).tolist()

# Filter the movie titles based on the IDs
title_list = movies_rating_tags_df[movies_rating_tags_df['movieId'].isin(titles_ids)]['title'].tolist()

# Print the titles
print(title_list)


['Fargo', 'Pretty Woman', 'Pinocchio']
